Importing

In [1]:
import torch
from torch.optim import Adam
from torch.optim import LBFGS
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import copy
import seaborn as sns
sns.set_style('darkgrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load Model Params

In [2]:
MODEL_NAME = "2D_Isotropic_Beam_6.51E-06_3000"
LOAD_PATH = "Models\\" + MODEL_NAME + ".tar"
checkpoint = torch.load(LOAD_PATH, map_location=device)
params_in, params_out, i_num, i_len = checkpoint['model_params']

Plotting Range

In [3]:
def grid(x,y,nx,ny):
    grid_arr = list()
    xs = torch.linspace(0,x,nx)
    ys = torch.linspace(0,y,ny)
    for x in xs:
        for y in ys:
            grid_arr.append((x.item(),y.item()))
    grid_arr = torch.tensor(grid_arr)
    return grid_arr

plotting = grid(10,10,20,20).requires_grad_(True).to(device)

In [4]:
# fn to create model
def pinn(input_size, output_size, num_layers, nodes_per_layer, activation):
    layers = []
    
    # Input layer
    layers.append(nn.Linear(input_size, nodes_per_layer))
    
    # Hidden layers
    for layer in range(num_layers):
        layers.append(nn.Linear(nodes_per_layer, nodes_per_layer))
        if layer == 0 and activation==Siren:
            layers.append(activation(l=0))
        else:
            layers.append(activation())
    
    # Output layer
    layers.append(nn.Linear(nodes_per_layer, output_size))
    
    return nn.Sequential(*layers)


# custom trainable Tanh activation fn
class TTanh(nn.Module):
    def __init__(self, features=i_len):
        super().__init__()
        self.features = features
        self.a = nn.Parameter(torch.randn(1, features))
        
    def forward(self, x):
        return torch.tanh(self.a * x)

#custom SIREN activation fn
class Siren(nn.Module):
    def __init__(self, features=i_len, l=1):
        super(Siren, self).__init__()
        self.features = features
        # Learnable parameters for the phase and bias
        if l: w = 1 
        else: w = 30
        n = torch.sqrt(torch.tensor(6.0/features))
        self.a_weight = nn.Parameter((torch.rand(1, features) *2*n - n)*w)
        self.a_bias = nn.Parameter(torch.randn(1, features))

    def forward(self, x):
        # Apply the periodic activation function
        return torch.sin( self.a_weight*x + self.a_bias)

In [5]:
model = pinn(params_in, params_out, i_num, i_len, TTanh)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

Solution

In [6]:
#sol?

Creating Position and Displacement Array

In [8]:
model_pred = list()
for i in range(len(plotting)):
    #x,y,dx,dy
    model_pred.append([plotting[i][0].item(), plotting[i][1].item(), model(plotting[i])[:,0].item(), model(plotting[i])[:,1].item()])
model_pred = np.array(model_pred)
LOAD_PATH = "Plot_Data\\" + MODEL_NAME + ".csv"
np.savetxt(LOAD_PATH, model_pred, delimiter=",")